In [ ]:
# Load datasets

import pandas as pd

datasets = {}
for name in ["BLCA", "BRCA", "COAD", "ESCA", "HNSC", "LGG", "LIHC", "LUAD", "LUSC", "PAAD", "PRAD", "SARC", "SKCM", "STAD", "UCEC"]:
    df = pd.read_csv(f'datasets/{name}.csv')
    datasets[name] = df
    vc = df.iloc[:,0].value_counts().sort_index()
    print(f"{name}\t: {df.shape}\t{list(vc)}")


In [2]:
# Dataset utils functions

from functools import reduce

def omics_str(omics):
    if type(omics) == str:
        omics = set([omics])
    return reduce(lambda a,b: a+"_"+b, omics)

def get_X(df, omics):
    if type(omics) == str:
        if omics == "__ALL__":
            return df.values
        omics = set([omics])
    indexes = [
        i
        for i, name in enumerate(df.columns)
        if name.split("_")[-1] in omics
    ]
    #print(f"selected {len(indexes)} for omics: {omics}")
    return df.values[:, indexes]

In [2]:
# Create estimators

from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.linear_model import Lasso


class MyLasso(Lasso):
    def __init__(self, C, max_iter=1000):
        self.C = C
        super().__init__(1.-C, max_iter=max_iter)
    def predict(self, X):
        return (super().predict(X) > 0.5).astype(float)

estimators = {
    "LR": (LogisticRegression, {'penalty':'l1', 'solver':'liblinear', 'max_iter':200000, 'random_state':0}),
    "SVM": (LinearSVC, {'penalty':'l1', 'dual':False, 'max_iter':200000, 'random_state':0}),
    "Lasso": (MyLasso, {'max_iter':2000})
}

In [4]:
# Tools for collecting results

from typing import List, Set, Dict
from statistics import mean

from sklearn.metrics import accuracy_score

from utils import stability_measure


class CVResult:
    def __init__(self, n_features:int):
        self.n_features = n_features # number of all features
        self.metrics = {
            "accuracy": accuracy_score
        }
        self.results = {}
        self.features_sets = []
    
    def add_feature_set(self, feature_set:Set[int]):
        self.features_sets.append(feature_set)

    def evaluate(self, prefix:str, y_real:List[float], y_predict:List[float]):
        for name, fun in self.metrics.items():
            key = f"{prefix}_{name}"
            if key not in self.results:
                self.results[key] = []
            self.results[key].append(fun(y_real,y_predict))
    
    def summary(self)->Dict[str,float]:
        results = {key:mean(values) for key, values in self.results.items()}
        results["fss_nogueira"] = stability_measure.nogueira(self.features_sets, self.n_features)
        results["fss_lustgarten"] = stability_measure.lustgarten(self.features_sets, self.n_features)
        results["fss_jaccard_index"] = stability_measure.jaccard_index(self.features_sets, self.n_features)
        results["avg_features"] = mean([len(s) for s in self.features_sets])
        results["all_features"] = self.n_features
        results["runs"] = len(self.features_sets)
        return results


In [ ]:
# Experiment core

from datetime import datetime
import re

from sklearn.model_selection import StratifiedKFold

from utils import estimator_helpers


data = {}
buffer_results = True

def append(data:Dict[str, List[float]], key:str, value:float):
    """Append new value to results collection"""
    if key not in data:
        data[key] = []
    data[key].append(value)

# experiment parameters
N_SPLITS = [5,]
OMICS = ["__ALL__", "cnv", "mirna", "mutation", "rna"]
RANDOM_SEEDS = [0,42,21]
FEATURES_FRACTION = [7,30,55] # percentage of the number of objects in the train data set

i = 0
I =  len(datasets) * len(OMICS) * len(N_SPLITS)

print(datetime.today().strftime('%Y-%m-%d %H:%M:%S'), "Experiment started ...")

for ds_name, df in datasets.items():
    print("------------------------------------")
    print(datetime.today().strftime('%Y-%m-%d %H:%M:%S'), f"Dataset {ds_name} {df.shape}")
    y = df.values[:,0]
    
    for omics in OMICS:
        print(datetime.today().strftime('%Y-%m-%d %H:%M:%S'), f"Omics {omics}")  
        X = get_X(df.iloc[:,1:], omics)

        for n_splits in N_SPLITS:
            train_size = round((1.-1./n_splits)*X.shape[0])
            features_nums = [round(train_size*fraction/100) for fraction in FEATURES_FRACTION]
            selectors = estimator_helpers.create_selectors(estimators, X, y, features_nums, train_size=train_size)
            print(datetime.today().strftime('%Y-%m-%d %H:%M:%S'), "Selectors with estimators created:")
            for name, selector in selectors.items():
                print(f"- {name}:", re.sub("\n\s*", " ", f"{selector.estimator}"))

            results = { name: CVResult(X.shape[1]) for name in selectors }

            for random_seed in RANDOM_SEEDS:
                skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_seed)
                    
                for train_index, test_index in skf.split(X, y):
                    X_train, y_train = X[train_index], y[train_index]
                    X_test, y_test = X[test_index], y[test_index]
                    
                    for name, selector in selectors.items():
                        # calculate results on this split for every estimator/selector
                        result = results[name]
                        selector.fit(X_train, y_train)
                        selected_features_indexes = selector.get_support(indices=True)
                        result.add_feature_set(set(selected_features_indexes))
                        result.evaluate("train", y_train, selector.estimator_.predict(X_train))
                        result.evaluate("test", y_test, selector.estimator_.predict(X_test))
            i+=1
            print(datetime.today().strftime('%Y-%m-%d %H:%M:%S'), f"Results {i}/{I} for {ds_name} {omics} and {n_splits}-fold cross validation")
            
            for name, result in results.items():
                r = result.summary()
                #print(f"{name:5} : ", r)
                append(data, "cv-folds", n_splits)
                append(data, "dataset", ds_name)
                append(data, "estimator", name)
                append(data, "estimator_details", re.sub("\n\s*", " ", f"{selectors[name].estimator_}"))
                append(data, "omics", omics_str(omics))
                for key, value in r.items():
                    append(data, key, value)

            if buffer_results:
                pd.DataFrame(data).to_csv(f"results_buffer.csv", index=False)

df = pd.DataFrame(data)
df.to_csv(f"results/r_{datetime.today().strftime('%Y%m%d_%H%M')}.csv", index=False)
print(datetime.today().strftime('%Y-%m-%d %H:%M:%S'), "Experiment finished ...")

In [ ]:
# For tests

from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split

from utils import estimator_helpers

class MyLasso(Lasso):
    def __init__(self, C):
        self.C = C
        super().__init__(1.-C)

df = datasets["BLCA"]
y = df.values[:,0]
X = get_X(df.iloc[:,1:], "__ALL__")

estimator_class = LinearSVC
estimator_params = {'penalty':'l1', 'dual':False, 'max_iter':100000}

X, _, y, _ = train_test_split(X, y, train_size=(1.-1./5), random_state=42, stratify=y)

#c_low = estimator_helpers.match_C_to_number_of_features(estimator_class, 31, X, y, estimator_params=estimator_params, verbose=True)
#print("--->", c_low)
#c_mid = estimator_helpers.match_C_to_number_of_features(estimator_class, 122, X, y, estimator_params=estimator_params, verbose=True)
#print("--->", c_mid)
c_high = estimator_helpers.match_C_to_number_of_features(estimator_class, 200, X, y, estimator_params=estimator_params, verbose=True)
print("--->", c_high)